In [0]:
# Step 2 bootstrap: run INSIDE /Workspace/Repos/<you>/surveytools
import os, textwrap, json, pathlib

# 1) Ensure we are in the repo folder (edit your username if different)
repo_dir = "/Workspace/Repos"
# Auto-detect the first child repo if user has only one; else set manually:
children = [p for p in pathlib.Path(repo_dir).glob("*/*") if p.is_dir()]
here = None
for p in children:
    if p.name == "surveytools":
        here = str(p); break
if here is None:
    raise SystemExit("Could not find /Workspace/Repos/<you>/surveytools — open this notebook INSIDE that folder.")
os.chdir(here)
print("Working in:", os.getcwd())

# 2) Create folder tree
os.makedirs("src/surveytools", exist_ok=True)

# 3) Write files
files = {
    "pyproject.toml": """\
[build-system]
requires = ["setuptools>=68", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "surveytools"
version = "0.1.0"
description = "Survey pipeline: Likert mapping UI → summaries → reporting (charts/PPTX) + CLI"
readme = "README.md"
requires-python = ">=3.9"
authors = [{ name = "Your Name" }]
dependencies = [
  "pandas>=1.5",
  "numpy>=1.24",
  "openpyxl>=3.1",
  "matplotlib>=3.7",
  "python-pptx>=0.6.21",
  "ftfy>=6.1; python_version >= '3.9'",
]
optional-dependencies = { widgets = ["ipywidgets>=8.0", "ipython>=8.0"] }

[project.scripts]
surveytools = "surveytools.cli:main"

[tool.pytest.ini_options]
pythonpath = ["src"]

[tool.ruff]
line-length = 100
""",
    "README.md": "# surveytools\n\nPipeline: Likert mapping UI → summaries → reporting (charts/PPTX) + CLI.\n",
    "src/surveytools/__init__.py": """\
from .summarize import TableSummarizer  # noqa: F401
# Optional UI imports guarded for environments without widgets
try:
    from .bulk_assistant import ExcelBulkAssistant  # noqa: F401
    from .report_assistant import ReportAssistant   # noqa: F401
    from .likert_mapper import LikertMapper         # noqa: F401
except Exception:  # widgets may be unavailable in headless contexts
    pass
""",
    # Minimal, compilable stubs so install works now; you can paste full versions later
    "src/surveytools/types.py": """\
from dataclasses import dataclass
@dataclass
class SummaryConfig:
    as_percent: bool = True
    decimals: int = 1
    output_mode: str = "percent"
    suppress_below_n: int = 30
    asterisk_from_n: int = 30
    asterisk_to_n: int = 49
    auto_group_scale: bool = True
    highlight_threshold: float = 5.0
""",
    "src/surveytools/constants.py": """\
ZC_TURQUOISE = "#3BD1BB"; ZC_CREAM = "#FCF5F1"; ZC_BEIGE = "#EEE2DB"; ZC_YELLOW = "#FAD565"
ZC_NIGHT = "#191824"; ZC_TEAL = "#148578"; ZC_MINT = "#79E8D7"; ZC_BLUE = "#7385FF"
ZC_WHITE = "#FFFFFF"; ZC_CORAL = "#FDA891"; ZC_BG = ZC_CREAM
ZERO_ALIASES = {"prefer not to say","prefer not to answer","don't know","do not know","unsure","not applicable","na","n/a","n.a.","none"}
PRESETS = {"Agreement (Strongly→Strongly)": [("strongly disagree",1),("disagree",2),("neutral",3),("agree",4),("strongly agree",5)]}
""",
    "src/surveytools/cleaning.py": """\
import pandas as pd
try:
    from ftfy import fix_text as _fix_text
except Exception:
    def _fix_text(x): return x
def fix_df_text(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy(); df.columns = [_fix_text(str(c)) for c in df.columns]
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].apply(lambda x: _fix_text(x) if isinstance(x, str) else x)
    return df
def fill_client_tokens(df: pd.DataFrame, col_name: str = "client_name") -> pd.DataFrame:
    if col_name not in df.columns or not df[col_name].notna().any(): return df
    df = df.copy(); val = str(df[col_name].dropna().iloc[0])
    df.columns = [str(c).replace("%client_name%", val) for c in df.columns]
    for c in df.select_dtypes(include="object").columns:
        df[c] = df[c].apply(lambda x: x.replace("%client_name%", val) if isinstance(x,str) else x)
    return df
""",
    "src/surveytools/summarize.py": """\
import re, ast, os
from typing import Optional, Dict, List, Tuple
import pandas as pd, numpy as np
from .types import SummaryConfig
class TableSummarizer:
    def __init__(self, df: pd.DataFrame, config: Optional[SummaryConfig]=None):
        self.df = df; self.config = config or SummaryConfig()
        self.group_options = [c for c in df.columns if str(c).strip().endswith("(demographics)") or c in ["polygon_name","age","ethnicities","education_group","gender","income_group"]]
        self.question_options = [c for c in df.columns if str(c).startswith("Q") and c not in self.group_options]
        self.summary_table: Optional[pd.DataFrame] = None
    def summarize_overall_and_demo(self, question: str, group_cols: List[str]) -> pd.DataFrame:
        # minimal placeholder: just return counts so the package installs/runs
        s = self.df[question].dropna().astype(str)
        counts = s.value_counts().sort_values(ascending=False).rename_axis("Option").reset_index(name="Count")
        counts.insert(0, "Row", "Overall"); counts.insert(1, "Group", "All respondents"); counts.rename(columns={"Count":"N"}, inplace=True)
        self.summary_table = counts; return counts
    def export_summary_table(self, filename: str, sheet_name: str):
        if self.summary_table is None: raise RuntimeError("No summary table.")
        with pd.ExcelWriter(filename, engine="openpyxl") as w:
            self.summary_table.to_excel(w, index=False, sheet_name=sheet_name)
""",
    "src/surveytools/likert_mapper.py": "class LikertMapper:\n    def __init__(self, df):\n        self.df = df\n        print('LikertMapper UI placeholder loaded.')\n",
    "src/surveytools/bulk_assistant.py": "class ExcelBulkAssistant:\n    def __init__(self, df, summarizer_class=None):\n        print('Bulk Assistant placeholder loaded.')\n",
    "src/surveytools/report_assistant.py": "class ReportAssistant:\n    def __init__(self, df, preselected_groups=None):\n        print('Report Assistant placeholder loaded.')\n",
    "src/surveytools/cli.py": """\
from __future__ import annotations
import argparse, re
import pandas as pd
from .cleaning import fix_df_text, fill_client_tokens
from .summarize import TableSummarizer
def parse_args(argv=None):
    p = argparse.ArgumentParser(prog="surveytools")
    p.add_argument("--input", required=True); p.add_argument("--output", required=True)
    p.add_argument("--groups", nargs="*", default=[]); p.add_argument("--question-prefix", default="Q")
    return p.parse_args(argv)
def main(argv=None) -> int:
    a = parse_args(argv)
    df = pd.read_excel(a.input) if a.input.lower().endswith("xlsx") else pd.read_csv(a.input)
    df = fill_client_tokens(fix_df_text(df))
    summ = TableSummarizer(df)
    questions = [c for c in df.columns if str(c).startswith(a.question-prefix) and c not in summ.group_options]
    with pd.ExcelWriter(a.output, engine="openpyxl") as w:
        for q in questions:
            res = summ.summarize_overall_and_demo(q, a.groups)
            sheet = re.sub(r"[\\\\/*?:\\[\\]]","", q)[:31]
            res.to_excel(w, index=False, sheet_name=sheet)
    print(f"Wrote: {a.output}"); return 0
"""
}

for path, content in files.items():
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        f.write(textwrap.dedent(content))
print("✅ Scaffolding written.")

# 4) Show tree
import subprocess, sys
subprocess.run(["bash","-lc","ls -R"])
print("\nNext: Git (top bar) → Commit → Push. Then run: %pip install -e '.[widgets]'")
